In [4]:
import csv, cv2, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Preprocess data
Merge images from three angles, left, center and right.

In [37]:
data = pd.read_csv('./data/driving_log.csv')

In [38]:
# Using three images with offset steering
offset_angle = 0.25

# Extract center image path and steering
df_center = data[['center','steering']]
# Extract left image path and steering
df_left = data[['left','steering']]
df_left.steering += offset_angle
# Extract right image path and steering
df_right = data[['right','steering']]
df_right.steering -= offset_angle

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [39]:
# unify columns to concat
df_center.columns = ['path', 'steering']
df_right.columns = ['path', 'steering']
df_left.columns = ['path', 'steering']
# concat these three dataframes into one
data0 = pd.concat([df_center, df_left,df_right],ignore_index =True)

In [40]:
data0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24108 entries, 0 to 24107
Data columns (total 2 columns):
path        24108 non-null object
steering    24108 non-null float64
dtypes: float64(1), object(1)
memory usage: 376.8+ KB


### Extra collecting data

In [41]:
def data_import(path):
    # The collected csv files don't have header, add them
    data = pd.read_csv(path + '/driving_log.csv', names = ['center','left','right','steering','throttle','brake','speed'])
    # Using three images with offset steering
    offset_angle = 0.25
    # Extract center image path and steering
    df_center = data[['center','steering']]
    # Extract left image path and steering
    df_left = data[['left','steering']]
    df_left.steering += offset_angle
    # Extract right image path and steering
    df_right = data[['right','steering']]
    df_right.steering -= offset_angle
    # unify columns to concat
    df_center.columns = ['path', 'steering']
    df_right.columns = ['path', 'steering']
    df_left.columns = ['path', 'steering']
    # concat these three dataframes into one
    df_result = pd.concat([df_center, df_left,df_right],ignore_index =True)
    return df_result

In [42]:
data1 = data_import('./data/run1')
data2 = data_import('./data/run2')
data3 = data_import('./data/run3')

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\pandas\core\generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [43]:
data_all = pd.concat([data0, data1, data2, data3],ignore_index =True)

In [79]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(data_all, test_size=0.2, random_state=42)

In [77]:
def generator(samples, batch_size=32):
    '''data generator'''
    num_samples = len(samples)
    while 1:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            angles = []
            for index in range(0, batch_size):
                # replace \\ to / for the collected data path
                batch_samples.iloc[index]['path'] = batch_samples.iloc[index]['path'].replace('\\','/')
                file_path = './data/IMG/' + batch_samples.iloc[index]['path'].split('/')[-1]
                image = cv2.imread(file_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                angle = float(batch_samples.iloc[index]['steering'])
                images.append(image)
                angles.append(angle)
#                 image_flipped = np.fliplr(center_image)
#                 angle_flipped = - center_angle
#                 images.append(image_flipped)
#                 angles.append(angle_flipped)
            
            X_train = np.array(images)
            y_train = np.array(angles)
            yield X_train, y_train

train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [26]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, Cropping2D

Using TensorFlow backend.


In [47]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1,1), 
                 padding='same', activation='relu'))
model.add(Conv2D(6, kernel_size=(3, 3), strides=(2,2), 
                 padding='same', activation='relu'))
model.add(Conv2D(16, kernel_size=(3, 3), strides=(2,2), 
                 padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(1))

In [48]:
model.compile(loss='mse', optimizer='adam')

In [49]:
epochs = 5
batch_size = 32
train_steps = len(train_samples)/batch_size
validation_steps = len(validation_samples)/batch_size

In [ ]:
history = model.fit_generator(train_generator, validation_data=validation_generator,
                              steps_per_epoch=train_steps,
                              validation_steps=validation_steps, epochs=5)

D:\Program Files\Anaconda3\envs\RoboND\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Epoch 1/5
 22/740 [..............................] - ETA: 2513s - loss: 0.0114

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

# plt.title('model loss')
plt.ylabel('loss', fontsize=11)
plt.xlabel('epoch', fontsize=11)
plt.legend(['train', 'valid'], loc='best')
plt.xlim((0,10))
plt.xticks(np.arange(0, 11, 2))
plt.grid()
# plt.savefig(img_dir + "/base_loss.png", dpi=300)
plt.show()

In [23]:
model.save('model.h5')